In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import torch
print("Available GPUs:", torch.cuda.device_count())
print("GPU Names:", [torch.cuda.get_device_name(i) for i in range(torch.cuda.device_count())])

from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

# Define a custom trainer to bypass loss calculation
class NoLabelTrainer(SFTTrainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        # Forward pass without loss calculation
        outputs = model(**inputs)
        loss = torch.tensor(0.0).to(model.device)
        return (loss, outputs) if return_outputs else None

max_seq_length = 2048
dtype = None
load_in_4bit = True
    
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "../Llama3_2_3B/", # choose "unsloth/Llama-3.2-1B-Instruct"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Available GPUs: 1
GPU Names: ['NVIDIA A100-PCIE-40GB']


/home/leeh65@ds.vumc.io/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth 2024.10.7: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A100-PCIE-40GB. Max memory: 39.431 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.4.0+cu121. CUDA = 8.0. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.27.post2. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.45s/it]
../Llama3_2_3B/ does not have a padding token! Will use pad_token = <|finetune_right_pad_id|>.
Unsloth: We fixed a gradient accumulation bug, but it seems like you don't have the latest transformers version!
Please update transformers, TRL and unsloth via:
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`
Unsloth 2024.10.7 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


In [2]:
from unsloth.chat_templates import get_chat_template
from unsloth.chat_templates import standardize_sharegpt
from datasets import Dataset
import pandas as pd

random_notes = pd.read_csv(f"../data/suicide_attempt/random_notes_Cleaned.csv")
random_notes = random_notes.dropna(subset=['note_text'])
random_notes = random_notes['note_text'].apply(lambda x: {"content": x}).tolist()
random_notes_dataset = Dataset.from_list(random_notes)

def tokenize_function(examples):
    return tokenizer(examples["content"], padding="max_length", truncation=True, max_length=512)

random_notes_dataset = random_notes_dataset.map(tokenize_function, batched=True)

def format_for_causal_lm(examples):
    examples['labels'] = examples['input_ids'].copy()
    return examples

random_notes_dataset = random_notes_dataset.map(format_for_causal_lm, batched=True)

print(random_notes_dataset)

Map: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30298/30298 [00:11<00:00, 2637.62 examples/s]

Dataset({
    features: ['content', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 30298
})


In [3]:
from unsloth.chat_templates import train_on_responses_only

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = random_notes_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 1,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs


In [4]:
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A100-PCIE-40GB. Max memory = 39.431 GB.
2.758 GB of memory reserved.


In [5]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 30,298 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 24,313,856


**** Unsloth: Please use our fixed gradient_accumulation_steps by updating transformers, TRL and Unsloth!
`pip install --upgrade --no-cache-dir unsloth git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git`


Step,Training Loss
1,2.839600
2,2.729600
3,2.661200
4,3.807700
5,2.644400
6,3.380100
7,2.579900
8,2.673300
9,3.143800
10,3.377300


In [6]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

97.8207 seconds used for training.
1.63 minutes used for training.
Peak reserved memory = 3.807 GB.
Peak reserved memory for training = 1.049 GB.
Peak reserved memory % of max memory = 9.655 %.
Peak reserved memory for training % of max memory = 2.66 %.


In [7]:
model_finetuned = model.merge_and_unload()
model_finetuned.tie_weights()
model_finetuned.save_pretrained("../Llama3_2_3B_FineTuned/llama-3.2-lora_model")
tokenizer.save_pretrained("../Llama3_2_3B_FineTuned/llama-3.2-lora_model")